# Create a hipscat catalog from gaia EDR3 distances

In [ ]:
from pathlib import Path

from dask.distributed import Client
from hipscat_import.catalog.arguments import ImportArguments
from hipscat_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hipscat_import.pipeline import pipeline_with_client

In [ ]:
gaia_root = Path("/ocean/projects/phy210048p/shared/hipscat/catalogs/gaia_dr3/")
catalog_name = "gaia_edr3_distances"
margin_name = f"{catalog_name}_10arcs"

### Catalog itself

In [ ]:
args = ImportArguments(
    ra_column="ra",
    dec_column="dec",
    use_hipscat_index=True,
    add_hipscat_index=False,
    input_path="./gedr3dist_joined_parquet_dataset",
    file_reader="parquet",
    output_artifact_name=margin_name,
    output_path=gaia_root,
)

with Client(n_workers=32) as client:
    display(client)
    pipeline_with_client(args, client)

### Margin catalog

In [ ]:
args = MarginCacheArguments(
    input_catalog_path=gaia_root / catalog_name,
    output_path=gaia_root,
    margin_threshold=10.0,
    output_artifact_name=margin_name,
)
with Client(n_workers=64) as client:
    display(client)
    pipeline_with_client(args, client)

### Test with lsdb

In [6]:
import lsdb

catalog = lsdb.read_hipscat(gaia_root / catalog_name, margin=gaia_root / margin_name)
catalog

,source_id,r_med_geo,r_lo_geo,r_hi_geo,r_med_photogeo,r_lo_photogeo,r_hi_photogeo,flag,ra,dec,Norder,Dir,Npix
npartitions=3243,,,,,,,,,,,,,
0,int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],uint8[pyarrow],uint64[pyarrow],uint64[pyarrow]
72057594037927936,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13817043656772681728,...,...,...,...,...,...,...,...,...,...,...,...,...
18446744073709551615,...,...,...,...,...,...,...,...,...,...,...,...,...
